In [1]:
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import scipy.stats as stats

import sys

In [2]:
# Set the maximum width of the display
pd.set_option('display.width', 1000)

# Set the maximum number of columns to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 30)

pd.set_option('display.precision', 2)

In [146]:
# fetch dataset
data = fetch_ucirepo(id=374)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data=data.data.features)

re_express = r'(\d{4})-(\d{2})-(\d{2})(\d{2}):(\d{2}):(\d{2})'

# Replace using the pattern
df['date'] = df['date'].str.replace(re_express, r'\1-\2-\3 \4:\5:\6', regex=True)

# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d %H:%M:%S", errors='raise')

# df['date'] = pd.date_range(start='2016-01-11 17:00:00', end='2016-05-27 18:00:00', freq='10min')

# If the data has target labels, you can also add them to the DataFrame
df['target'] = data.data.targets

# add ID as column
df['ID'] = np.arange(1, len(df) + 1, 1)

# Set the 'date' column as the index
df.set_index('date', inplace=True)

# Optionally, sort the DataFrame by the new datetime index
df.sort_index(inplace=True)

# since we sort the dataframe by the new datetime index
df = df.reset_index()

# Display the first few rows of the DataFrame
df.head()

,date,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,target,ID
0,2016-01-11 17:00:00,30,19.89,47.60,19.2,44.79,19.79,44.73,19.00,45.57,17.17,55.20,7.03,84.26,17.20,41.63,18.2,48.90,17.03,45.53,6.60,733.5,92.0,7.00,63.00,5.3,13.28,13.28,60,1
1,2016-01-11 17:10:00,30,19.89,46.69,19.2,44.72,19.79,44.79,19.00,45.99,17.17,55.20,6.83,84.06,17.20,41.56,18.2,48.86,17.07,45.56,6.48,733.6,92.0,6.67,59.17,5.2,18.61,18.61,60,2
2,2016-01-11 17:20:00,30,19.89,46.30,19.2,44.63,19.79,44.93,18.93,45.89,17.17,55.09,6.56,83.16,17.20,41.43,18.2,48.73,17.00,45.50,6.37,733.7,92.0,6.33,55.33,5.1,28.64,28.64,50,3
3,2016-01-11 17:30:00,40,19.89,46.07,19.2,44.59,19.79,45.00,18.89,45.72,17.17,55.09,6.43,83.42,17.13,41.29,18.1,48.59,17.00,45.40,6.25,733.8,92.0,6.00,51.50,5.0,45.41,45.41,50,4
4,2016-01-11 17:40:00,40,19.89,46.33,19.2,44.53,19.79,45.00,18.89,45.53,17.20,55.09,6.37,84.89,17.20,41.23,18.1,48.59,17.00,45.40,6.13,733.9,92.0,5.67,47.67,4.9,10.08,10.08,60,5


In [4]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, RobustScaler

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    mean_squared_error,
    root_mean_squared_error
)

In [5]:
import xgboost as xgb

from hyperopt import (
    fmin,  # Function for minimizing/maximizing an objective function
    tpe,   # Tree-structured Parzen Estimator (TPE) algorithm for optimization
    hp,    # Defines search space for hyperparameters
    STATUS_OK,  # Constant indicating successful completion of an objective
    Trials  # Container for storing results of each trial
)


from hyperopt.pyll import scope # Handles scoping in hyperparameter definitions
from hyperopt.pyll.stochastic import sample

from functools import partial

In [6]:
X = df.drop(['target', 'date'], axis=1 )
y = df['target']

X.columns

Index(['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2'], dtype='object')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

len(X_train), len(y_train), len(X_test), len(y_test)

(13814, 13814, 5921, 5921)

In [227]:
# https://sklearn-template.readthedocs.io/en/latest/user_guide.html

# Custom transformer to create multiple lags
class MultiLagFeature(BaseEstimator, TransformerMixin):
    def __init__(self, max_lag: int = 10):
        self.max_lag = max_lag
        self.column = None
    
    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            self.column = X.columns  # Set column name from the DataFrame
        else:
            raise ValueError("Input must be a pandas DataFrame")
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame")
        
        if self.column is None:
            raise ValueError("Column name not set. Make sure to call fit before transform.")
        
        # Create a list to hold lagged DataFrames
        lagged_frames = []
        
        for column in self.column:
            # For each column, create lagged features and store them in a DataFrame
            lagged_data = pd.DataFrame({
                f'{column}_lag{lag}': X[column].shift(lag)
                for lag in range(1, self.max_lag + 1)
            })
            lagged_frames.append(lagged_data)
        
        # Concatenate all lagged DataFrames with the original DataFrame
        X_lagged = pd.concat([X] + lagged_frames, axis=1)
        self.feature_names_ = X_lagged.columns
        return X_lagged
    
    def get_feature_names_out(self, *args, **params):
        return self.feature_names_
    

class SineTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, season_length: list[int] = 1):
        self.column = None
        self.season_length = [season_length] if isinstance(season_length, int) else season_length
    
    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            self.column = X.columns  # Set column name from the DataFrame
        else:
            raise ValueError("Input must be a pandas DataFrame")
        return self
    
    def transform(self, X):
        vector_names = [f'Sine_{length}' for length in self.season_length]
        W = pd.DataFrame(columns=vector_names, index=range(X.shape[0]))
        for column in self.column:
            for season in self.season_length:
                W[f'Sine_{season}'] = np.sin(2 * np.pi * X[column] * (1/season))
        self.feature_names_ = W.columns
        return W
    
    def get_feature_names_out(self, *args, **params):
        return self.feature_names_

class CosineTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, season_length: list[int] = 1):
        self.column = None
        self.season_length = [season_length] if isinstance(season_length, int) else season_length
    
    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            self.column = X.columns  # Set column name from the DataFrame
        else:
            raise ValueError("Input must be a pandas DataFrame")
        return self
    
    def transform(self, X):
        vector_names = [f'Cosine_{length}' for length in self.season_length]
        W = pd.DataFrame(columns=vector_names, index=range(X.shape[0]))
        for column in self.column:
            for season in self.season_length:
                W[f'Cosine_{season}'] = np.cos(2 * np.pi * X[column] * (1/season))
        self.feature_names_ = W.columns
        return W
    
    def get_feature_names_out(self, *args, **params):
        return self.feature_names_


In [228]:

trip_distance_trans = ColumnTransformer(
    transformers=[
        ("transform",  MultiLagFeature(max_lag=144), ['target', 'RH_8', 'RH_out']),
        ('Sin', SineTransformer(season_length = 144), ['ID'] ),
        ('Cos', CosineTransformer(season_length = 144), ['ID'])
    ],
    # remainder='passthrough'
).set_output(transform='pandas')

In [229]:
trip_distance_trans

ColumnTransformer(transformers=[('transform', MultiLagFeature(max_lag=144),
                                 ['target', 'RH_8', 'RH_out']),
                                ('Sin', SineTransformer(season_length=[144]),
                                 ['ID']),
                                ('Cos', CosineTransformer(season_length=[144]),
                                 ['ID'])])

In [230]:
X = trip_distance_trans.fit_transform(df)

In [231]:
X

,transform__target,transform__RH_8,transform__RH_out,transform__target_lag1,transform__target_lag2,transform__target_lag3,transform__target_lag4,transform__target_lag5,transform__target_lag6,transform__target_lag7,transform__target_lag8,transform__target_lag9,transform__target_lag10,transform__target_lag11,transform__target_lag12,transform__target_lag13,transform__target_lag14,transform__target_lag15,transform__target_lag16,transform__target_lag17,transform__target_lag18,transform__target_lag19,transform__target_lag20,transform__target_lag21,transform__target_lag22,transform__target_lag23,transform__target_lag24,transform__target_lag25,transform__target_lag26,transform__target_lag27,transform__target_lag28,transform__target_lag29,transform__target_lag30,transform__target_lag31,transform__target_lag32,transform__target_lag33,transform__target_lag34,transform__target_lag35,transform__target_lag36,transform__target_lag37,transform__target_lag38,transform__target_lag39,transform__target_lag40,transform__target_lag41,transform__target_lag42,transform__target_lag43,transform__target_lag44,transform__target_lag45,transform__target_lag46,transform__target_lag47,transform__target_lag48,transform__target_lag49,transform__target_lag50,transform__target_lag51,transform__target_lag52,transform__target_lag53,transform__target_lag54,transform__target_lag55,transform__target_lag56,transform__target_lag57,transform__target_lag58,transform__target_lag59,transform__target_lag60,transform__target_lag61,transform__target_lag62,transform__target_lag63,transform__target_lag64,transform__target_lag65,transform__target_lag66,transform__target_lag67,transform__target_lag68,transform__target_lag69,transform__target_lag70,transform__target_lag71,transform__target_lag72,transform__target_lag73,transform__target_lag74,transform__target_lag75,transform__target_lag76,transform__target_lag77,transform__target_lag78,transform__target_lag79,transform__target_lag80,transform__target_lag81,transform__target_lag82,transform__target_lag83,transform__target_lag84,transform__target_lag85,transform__target_lag86,transform__target_lag87,transform__target_lag88,transform__target_lag89,transform__target_lag90,transform__target_lag91,transform__target_lag92,transform__target_lag93,transform__target_lag94,transform__target_lag95,transform__target_lag96,transform__target_lag97,transform__target_lag98,transform__target_lag99,transform__target_lag100,transform__target_lag101,transform__target_lag102,transform__target_lag103,transform__target_lag104,transform__target_lag105,transform__target_lag106,transform__target_lag107,transform__target_lag108,transform__target_lag109,transform__target_lag110,transform__target_lag111,transform__target_lag112,transform__target_lag113,transform__target_lag114,transform__target_lag115,transform__target_lag116,transform__target_lag117,transform__target_lag118,transform__target_lag119,transform__target_lag120,transform__target_lag121,transform__target_lag122,transform__target_lag123,transform__target_lag124,transform__target_lag125,transform__target_lag126,transform__target_lag127,transform__target_lag128,transform__target_lag129,transform__target_lag130,transform__target_lag131,transform__target_lag132,transform__target_lag133,transform__target_lag134,transform__target_lag135,transform__target_lag136,transform__target_lag137,transform__target_lag138,transform__target_lag139,transform__target_lag140,transform__target_lag141,transform__target_lag142,transform__target_lag143,transform__target_lag144,transform__RH_8_lag1,transform__RH_8_lag2,transform__RH_8_lag3,transform__RH_8_lag4,transform__RH_8_lag5,transform__RH_8_lag6,transform__RH_8_lag7,transform__RH_8_lag8,transform__RH_8_lag9,transform__RH_8_lag10,transform__RH_8_lag11,transform__RH_8_lag12,transform__RH_8_lag13,transform__RH_8_lag14,transform__RH_8_lag15,transform__RH_8_lag16,transform__RH_8_lag17,transform__RH_8_lag18,transform__RH_8_lag19,transform__RH_8_lag20,transform__RH_8_lag21,tr